In [1]:
import pandas

Rdf = pandas.read_excel('pure_data.xlsx', sheet_name='Ark1')
print(Rdf.shape)

(304, 5)


In [2]:
Rdf.head(2)

,Input story,Statement,principle,number,yes/no
0,My name is Ronald Smith and I hit a guy. I am ...,"Your Honor, I stand before you today to take f...",emotions,0,yes
1,My name is Ronald Smith and I hit a guy. I am ...,"Your Honor, I stand before you today to take f...",coherence,1,yes


In [3]:
from info import yesnoCR_turned

# yesnoCR_turned = [
#     """Is anything in the statement badly phrased or difficult to follow?""",
#     # """repeat""",
#     # """unreasonable?""",
#     """Does the statement ignore the importance of the defendant's guilty plea (an important mitigating factor)?""",
#     """Does the statement give a different impression (compared to the input story) of whether the defendant has already suffered as a consequence of their crime? We don't consider emotional stress or what may happen in the future as suffering.""",
#     #"""Does the statement give a different impression of how the defendant has cooperated with authorities (for example providing them with information), than the input story?""",
#     """Does the statement break the rule 'only mention the defendant's age if it is outside of the interval 30-79'?""",
#     """Does the statement and input story seem to disagree on whether the defendant has a prior criminal record?""",# AND If the defendant seems to have no previous record according to the input story, is this properly mentioned in the statement?""",
#     # """provocation"""
#     """Does the statement fail to mention any mental illness or disability (if the defendant has this)?""",
#     """Does the statement miss or add important parts of explaining the offense (and the defendant's involvement in it), compared to the input story?""",
#     # amends
#     """Does the statement give a different impression than the input story of whether the defendant has or want to make amends or 'make up' for the crime?""",
#     """Does the statement give a different impression than the input story of whether the defendant has or wants to self-improve (eg counselling) or prevent the offense from happening again?""",
#     """Does the statement give a different impression than the input story of the defendant's good character (for example contributing to the community in some way)?""",
#     """Does the statement contain any misplaced information for a plea in mitigation (eg mentioning the 'input story' or another statement)?""",
#     """Does the statement overemphasize, or show emotions different from the input story?""",# Or does it show emotions that are not implied by the input story?""",
#     """Is there any information in the statement that is unsupported by the input story?""",# (for example about the defendant's job or future intentions)?""",
# ]

In [4]:
fromto_excel_real = {"emotions": 11, "coherence": 0, "reasonable": 0, "explain offense": 6, "guilty plea": 1, "suffering": 2, "age": 3, "criminal record": 4, "mental": 5, "amends": 7, "self-improvement": 8, "character": 9, "irrelevant": 10, "hallusination": 12}
def swap_numbers(principle):
    if principle in fromto_excel_real:
        num = fromto_excel_real[principle]
    else:
        num = 999
    return num

Rdf["New numbers"] = Rdf["principle"].apply(swap_numbers)
Rdf.drop(Rdf.index[Rdf["New numbers"] == 999], inplace = True)
Rdf = Rdf.drop('number', axis=1)

#below = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n"
yesnoWhy = " Please answer 'yes' or 'no'. "
diff = "Make sure to think carefully about what is said in the input story VS what is said in the statement."

from info import yesnoCR_turned as yesnoCR

def apply_yesno(numb):
    return yesnoCR[numb]

def reverse_yesno(yn):
    if yn.lower() == "yes":
        return "no"
    elif yn.lower() == "no":
        return "yes"
    else:
        return "unvalid"

#Rdf["DFyesnoCR  = Rdf["New numbers .apply(apply_yesno)
Rdf["instruction"] = Rdf["New numbers"].apply(apply_yesno) + yesnoWhy + diff
Rdf["input"] = "### INPUT STORY: " + Rdf["Input story"] + "\n### STATEMENT: " + Rdf["Statement"]
Rdf["output"] = Rdf["yes/no"].apply(reverse_yesno)
#Rdf.rename(columns = {'yes/no':'output'}, inplace = True)

cols = ["instruction", "input", "output"]
Rdf = Rdf[cols]
print(Rdf.shape)

(281, 3)


In [5]:
print(Rdf[Rdf["output"]=="unvalid"])

Empty DataFrame
Columns: [instruction, input, output]
Index: []


In [6]:
Rdf.head(3)

,instruction,input,output
0,"Does the statement overemphasize, or show emot...",### INPUT STORY: My name is Ronald Smith and I...,no
1,Is anything in the statement badly phrased or ...,### INPUT STORY: My name is Ronald Smith and I...,no
2,Does the statement give a different impression...,### INPUT STORY: My name is Ronald Smith and I...,yes


In [7]:
json_str = Rdf.to_json('data_file1.json', orient = 'records')

# Dataset w full prompts

In [8]:
Pdf = Rdf.copy()
system_prompt="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."
Pdf["prompt"] = system_prompt + " USER: " + Rdf["instruction"] + "\n" + Rdf["input"] + " ASSISTANT: "
Pdf["labels"] = Pdf["output"]
Pdf = Pdf[['prompt', 'labels']]
Pdf

,prompt,labels
0,A chat between a curious user and an artificia...,no
1,A chat between a curious user and an artificia...,no
2,A chat between a curious user and an artificia...,yes
3,A chat between a curious user and an artificia...,yes
4,A chat between a curious user and an artificia...,no
...,...,...
299,A chat between a curious user and an artificia...,yes
300,A chat between a curious user and an artificia...,yes
301,A chat between a curious user and an artificia...,yes
302,A chat between a curious user and an artificia...,yes


In [9]:
json_str = Pdf.to_json('data_file2.json', orient = 'records')